In [12]:
import pandas as pd
import os
import json
from collections import defaultdict

In [5]:

with open('../config.json') as config_file:
    config = json.load(config_file)
data_path = config['data_path']
print(data_path)

/scratch2/hle


In [6]:
df = pd.read_csv(os.path.join(data_path,f'refCOCO/test/attr_tables_with_target_box/attr_1.tsv'), encoding='utf-8',sep='\t')

In [7]:
df.head()

,box_alias,image_id,ann_id,ref_id,salience,x1,y1,w,h,TYPE_yolk,...,ATTR_thin,ATTR_decorative,ATTR_wrinkled,ATTR_peeling,ATTR_golden,ATTR_metallic,ATTR_back,ATTR_thick,ATTR_black and white,ATTR_leaning
0,cat-1,169820,46375,35499,0.096320,29.428690,82.913445,116.503242,137.654572,0.000624,...,0.002494,0.002493,0.002493,0.002493,0.002494,0.002493,0.002499,0.002495,0.002503,0.002494
1,tail-1,169820,46375,35499,0.019117,440.889801,153.144791,54.440521,58.466034,0.000624,...,0.002497,0.002493,0.002495,0.002493,0.002494,0.002493,0.002524,0.002508,0.002497,0.002493
2,cat-2,169820,46375,35499,0.297689,182.072052,84.467247,255.985809,193.624573,0.000624,...,0.002494,0.002494,0.002495,0.002493,0.002494,0.002493,0.002495,0.002495,0.002496,0.002494
3,head-1,169820,46375,35499,0.053074,30.695189,70.197906,96.414406,91.655350,0.000624,...,0.002495,0.002494,0.002494,0.002493,0.002494,0.002493,0.002505,0.002496,0.002509,0.002494
4,pillow-1,169820,46375,35499,0.163228,61.365448,2.605297,248.645721,109.302063,0.000624,...,0.002494,0.002500,0.002497,0.002493,0.002493,0.002493,0.002494,0.002495,0.002495,0.002493


In [45]:
types = [col[5:] for col in df.columns if 'TYPE_' in col]
attributes = [col[5:] for col in df.columns if 'ATTR_' in col]

In [55]:
theta_type = 0.0007
theta_att = 0.0025
top_k_features = 1

dict_obj_to_types_above_threshold = defaultdict(list)
obj_to_types = defaultdict(dict)
for t in types:
    col_type = f'TYPE_{t}'
    type_df = df[['box_alias', col_type]]
    for index, row in type_df.iterrows():
        obj = row.iloc[0]
        prob = row.iloc[1]
        if prob > theta_type:
            dict_obj_to_types_above_threshold[obj].append((prob, t))
            
# for each object sort the features (based on prob) then pick top_k_features
for obj in dict_obj_to_types_above_threshold:
    dict_obj_to_types_above_threshold[obj].sort(reverse=True)
    for i in range(min(top_k_features, len(dict_obj_to_types_above_threshold[obj]))):
        t = dict_obj_to_types_above_threshold[obj][i][1]
        obj_to_types[obj][t] = 1
            
dict_obj_to_attrs_above_threshold = defaultdict(list)
obj_to_attributes = defaultdict(dict)
for att in attributes:
    col_att = f'ATTR_{att}'
    attr_df = df[['box_alias', col_att]]
    for index, row in attr_df.iterrows():
        obj = row.iloc[0]
        prob = row.iloc[1]
        if prob > theta_att:
            dict_obj_to_attrs_above_threshold[obj].append((prob, att))
            
# for each object sort the features (based on prob) then pick top_k_features
for obj in dict_obj_to_attrs_above_threshold:
    dict_obj_to_attrs_above_threshold[obj].sort(reverse=True)
    for i in range(min(top_k_features, len(dict_obj_to_attrs_above_threshold[obj]))):
        att = dict_obj_to_attrs_above_threshold[obj][i][1]
        obj_to_attributes[obj][att] = 1

In [56]:
obj_to_types

defaultdict(dict,
            {'carpet-1': {'carpet': 1},
             'cat-1': {'cat': 1},
             'cat-2': {'cat': 1},
             'cat-3': {'cat': 1},
             'couch-1': {'couch': 1},
             'dog-1': {'cat': 1},
             'floor-1': {'OOV': 1},
             'head-1': {'head': 1},
             'pillow-1': {'pillow': 1},
             'table-1': {'table': 1},
             'tail-1': {'tail': 1}})

In [57]:
dict_obj_to_types_above_threshold

defaultdict(list,
            {'carpet-1': [(0.0008227874641306697, 'carpet'),
              (0.00079056725371629, 'OOV')],
             'cat-1': [(0.0014996061800047755, 'cat')],
             'cat-2': [(0.0012565188808366654, 'cat')],
             'cat-3': [(0.0009905402548611164, 'cat')],
             'couch-1': [(0.0008597172563895582, 'couch')],
             'dog-1': [(0.000996065908111632, 'cat'),
              (0.0007984827971085906, 'dog')],
             'floor-1': [(0.0009766542352735996, 'OOV'),
              (0.0007906624232418835, 'floor')],
             'head-1': [(0.0010496871545910835, 'head'),
              (0.0007322388701140881, 'OOV')],
             'pillow-1': [(0.0008735843002796173, 'pillow'),
              (0.0007227418827824295, 'OOV')],
             'table-1': [(0.0007636789814569057, 'table')],
             'tail-1': [(0.0013583543477579953, 'tail')]})

In [58]:
obj_to_attributes

defaultdict(dict,
            {'carpet-1': {'white': 1},
             'cat-1': {'black': 1},
             'cat-2': {'sleeping': 1},
             'cat-3': {'black': 1},
             'couch-1': {'white': 1},
             'dog-1': {'sleeping': 1},
             'floor-1': {'gray': 1},
             'head-1': {'black': 1},
             'pillow-1': {'white': 1},
             'table-1': {'black': 1},
             'tail-1': {'black': 1}})

In [59]:
dict_obj_to_attrs_above_threshold

defaultdict(list,
            {'carpet-1': [(0.0034836912527680397, 'white'),
              (0.002927228109911084, 'gray'),
              (0.002705104416236281, 'brown'),
              (0.002600763225927949, 'tan'),
              (0.0025901973713189363, 'beige'),
              (0.002561704255640507, 'black'),
              (0.002555958228185773, 'wooden'),
              (0.002538253320381045, 'large'),
              (0.002536132698878646, 'small'),
              (0.002522653667256236, 'clean'),
              (0.0025220683310180907, 'open'),
              (0.0025174294132739305, 'wood'),
              (0.002515915315598249, 'long'),
              (0.002514723921194673, 'carpeted'),
              (0.002514582592993975, 'blue'),
              (0.002512936713173985, 'silver'),
              (0.002512248232960701, 'fluffy'),
              (0.0025111539289355282, 'dark'),
              (0.0025084617082029577, 'big'),
              (0.0025078230537474155, 'laying'),
              (0.002506766